# Sensitivity and Complementary Sensitivtiy Functions
## today's objectives: 
- recall  main advantages of feedback
- derive and understand main feedback equation

## Assume:
- transfer function representation of a system
- working knowledge of bode plots

# Notation

![openloop](openloop.png "Open-Loop")

- $P$: plant (system)
- $u$: input (variable can be manipulated)
- $y$: output (variable to be controlled)
- $d$: disturbance (affects value of output)

![feedbackloop](feedbackloop.png "Feedback")

- $K$: controller
- $r$: reference signal (desired output)
- $n$: measurement noise (error in measurement)
- $y_m$: measured output


# Why feedback?










![AttaboticsThunderdome](attabotics-thunderdome.png "Attabotics Thunderdome")

- Ability to follow desired signals
- Stability (design)
- Reduction of effects of process noise / load disturbances (noise rejection, compensation)
- Reduction of effects of model uncertainties
- Efficiency (design)

In [ ]:
import control as ct
import numpy as np
import controls_teaching_lecture_utilities as cu

In [ ]:
# Very simple model of a car (input = gas pedal, output = speed of car)
s = ct.tf('s')
P = 1/(s+1)

# Controller (for cruise control)
K = 5*(s+1)/s

# Reference, disturbance and measurement noise signals
t = np.linspace(0, 50, 101)
r = [0 if ti <= 5 else
     80 if ti <= 30 else
     100 for ti in t]
r = np.array(r)
d = np.zeros(101)
n = np.random.randn(101)

# simulate closed-loop and open-loop models for selected reference, disturbance and measurement noise signals
y_closed_loop, u_closed_loop = cu.sim_closed_loop(P,K,t,r,d,n)
y_open_loop = cu.sim_open_loop(P,t,r,d)

# plot results
cu.make_open_and_closed_loop_plots(t,r,d,n,y_closed_loop,y_open_loop)

# Effect of disturbance (open-loop vs closed-loop)

In [ ]:
d_hill = 10*np.sin(t/100*np.pi*4)
y_closed_loop, u_closed_loop = cu.sim_closed_loop(P,K,t,r,d_hill,n)
y_open_loop = cu.sim_open_loop(P,t,r,d_hill)

cu.make_open_and_closed_loop_plots(t,r,d_hill,n,y_closed_loop,y_open_loop)

# Effect of uncertainty in the model (open-loop vs closed-loop)

In [ ]:
P_changed = 0.95/(s+1)
y_closed_loop, u_closed_loop = cu.sim_closed_loop(P_changed,K,t,r,d,n)
y_open_loop = cu.sim_open_loop(P_changed,t,r,d)
    
cu.make_open_and_closed_loop_plots(t,r,d,n,y_closed_loop,y_open_loop)

# The Feedback Equation!

![feedbackloop](feedbackloop.png "Feedback")

$$
\begin{align}
y(s) &= P(s) \Big( u(s) + d(s) \Big) \\
&= P(s)K(s) \Big( r(s) - y_m(s) \Big ) + P(s)d(s) \\
&= P(s)K(s) r(s) - P(s)K(s) y_m(s) + P(s)d(s) \\
&= P(s)K(s)r(s) - P(s)K(s)\Big(y(s) + n(s)\Big) + P(s)d(s) \\
&= P(s)K(s)r(s) - P(s)K(s)y(s)  - P(s)K(s) n(s)+ P(s)d(s) \\
&= \frac{P(s)K(s)}{1+P(s)K(s)} r(s) - \frac{P(s)K(s)}{1+P(s)K(s)}n(s) + \frac{P(s)}{1+P(s)K(s)}d(s)
\end{align}
$$

Loop transfer function:

$$L(s) = P(s)K(s)$$

Sensitivity Function:

$$S(s) = \frac{1}{1+P(s)K(s)} = \frac{1}{1 + L}$$ 

Complementary Sensitivity Function:

$$T(s) = \frac{P(s)K(s)}{1+P(s)K(s)} = \frac{L}{1+L}$$ 

Interesting property:

$$S(s) + T(s) = 1$$

Feedback equation becomes:

$$y(s) = T(s)r(s) - T(s)n(s) + S(s)P(s)d(s)$$

In terms of tracking error:

$$
\begin{align}
\varepsilon(s) &= r(s) - y(s) \\
&= r(s) -  T(s)r(s) - T(s)n(s) - S(s)P(s)d(s) \\
&= S(s)r(s) + T(s)n(s) - S(s)P(s)d(s)
\end{align}
$$

Observations:
- $S(s)$ determines the error due to not tracking $r$ 
- $S(s)$ determines the error due to process disturbances $d$
- $T(s)$ determines the error due to measurement noise $n$

# We want both $S(s)$ and $T(s)$ to be small!!
# However recall $S(s) + T(s) = 1$. 


# Let's design $K$! (Idea 1)

There are several possibilities to obtain this desired behaviour:

choose $K$ as a (large) constant.

$$\varepsilon(s) = \frac{1}{1+K(s)P(s)}r(s) - \frac{K(s)P(s)}{1+K(s)P(s)}n(s) - \frac{P(s)}{1+K(s)P(s)}d(s)$$

Future topic: PID control

# Your Turn!

In [ ]:
import control as ct
import numpy as np
import controls_teaching_lecture_utilities as cu
s = ct.tf('s')

# Reference, disturbance and measurement noise signals
t = np.linspace(0, 50, 101)
r = [0 if ti <= 5 else
     80 if ti <= 30 else
     100 for ti in t]
r = np.array(r)
d = np.zeros(101)
n = np.random.randn(101)

# Liquid levels example from "Modern Control Engineering ", by Katsuhiko Ogata 

R = 100 #sec/m^2
C = 1/5 #m^2 (cross sectional area of tank)

P_liquid_level = 1/(R*C*s + 1)

# try different controllers
K = 50


y, u = cu.sim_closed_loop(P_liquid_level,K,t,r,d,n)
cu.make_closed_loop_plots(t,r,d,n,y,u)

# Assignment

For each of the following 3 systems:
1. Design 2 possible controllers (using what you learned in this lecture)
2. Choose 3 different scenarios (different disturbances, reference signals, levels of measurement noise)
3. Simulate the different senarios
4. Evaluate/analyze their performance based on the output and input of the plant.
5. Make a recommendation as to which of the two selected controllers is better

Main objective of assignment is to develop insight into the feedback equation, and how it is tied to the control objectives, not to design a good controller.

![question-hydraulic](question-hydraulic.png "Question Hydraulic")

In [ ]:
# Liquid levels example from "Modern Control Engineering " by Katsuhiko Ogata, page 93
P_liquid_level = 1/(R*C*s + 1)

![question-mech](question-mech.png "Question Mechanical")

In [ ]:
# Mechanical system example from "Modern Control Engineering " by Katsuhiko Ogata, page 83
P_mechanical = (b*s + k)/(m*s**2 + b*s + k)

![question-elec](question-elec.png "Question Electrical")

In [ ]:
# Electrical system example from "Modern Control Engineering " by Katsuhiko Ogata, page 90
P_electrical = 1/(R1*C1*R2*C2*s**2 + (R1*C1 + R2*C2 + R1*C2)*s + 1)

# Material for next lecture

# Balancing Priorities

$$\varepsilon(s) =S(s)r(s) + T(s)n(s) - S(s)P(s)d(s)$$

![FrequencyDomainRepresentationDesiredSensitivity](sensitivityBode.png "Frequency Domain Representation of Desirable Sensitivity Function")

# That is great insight!
# However, how does this help us? Our ''tuning knob'' is the controller K...

In [ ]:
L = 1/s

S = 1/(1+L)
T = L/(1+L)
mag, phase, omega = bode([S, T], dB=True, plot=False)

fig, ax = plt.subplots(1,1, figsize = [10,5])
ax.semilogx(omega[0],[20*math.log10(x) for x in mag[0]], label='Sensitivity Function')
ax.semilogx(omega[1],[20*math.log10(x) for x in mag[1]], label='Complementary Sensitivity Function')
#ax.semilogx(omega[0],mag[0], label='Sensitivity Function')
#ax.semilogx(omega[1],mag[1], label='Complementary Sensitivity Function')
ax.set_xlabel('Frequency (rad/s)')
ax.set_ylabel('Magnitude (dB)')
ax.set_title('Bode Plot')
plt.legend(bbox_to_anchor=(1.05,1), borderaxespad=0)
plt.show()

# How does $K$ affect frequency response of $S$ and $T$?

$$
\begin{align}
S(s) &= \frac{1}{1+L(s)}  = \frac{1}{1+P(s)K(s)} \\
T(s) &= \frac{L(s)}{1+L(s)} = \frac{P(s)K(s)}{1+P(s)K(s)}  
\end{align}
$$

In [ ]:
L = 1/s

S = 1/(1+L)
T = L/(1+L)
mag, phase, omega = bode([S, T, L], dB=True, plot=False)



fig, ax = plt.subplots(1,1, figsize = [10,5])
ax.semilogx(omega[0],[20*math.log10(x) for x in mag[0]], label='Sensitivity Function')
ax.semilogx(omega[1],[20*math.log10(x) for x in mag[1]], label='Complementary Sensitivity Function')
ax.semilogx(omega[2],[20*math.log10(x) for x in mag[2]], label='Loop Transfer Function')
#ax.semilogx(omega[0],mag[0], label='Sensitivity Function')
#ax.semilogx(omega[1],mag[1], label='Complementary Sensitivity Function')
ax.set_xlabel('Frequency (rad/s)')
ax.set_ylabel('Magnitude (dB)')
ax.set_title('Bode Plot')
plt.legend(bbox_to_anchor=(1.05,1), borderaxespad=0)
#ax.grid(True)
plt.show()

![FrequencyDomainRepresentationDesiredLoop](desiredLoopTF.png "Frequency Domain Representation of Desirable Loop Function")

# Let's design $K$! (Idea 2)

![FrequencyDomainRepresentationDesiredLoop](desiredLoopTF.png "Frequency Domain Representation of Desirable Loop Function")

One possible loop transfer function that has our desired behaviour:

$$L(s) = \frac{1}{s}$$

(recall that this is the TF of an integrator)

But 
$$L(s) = K(s)P(s)$$
so can we solve for K?

Future topic: pole placement